In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from pmdarima.arima import arima
import scipy
import seaborn as sns
import sklearn
import statsmodels.api as sm
import yfinance 
import warnings 
warnings.filterwarnings("ignore")
sns.set()


# Load data 

In [4]:
raw_data = yfinance.download(tickers="^GSPC ^FTSE ^N225 ^GDAXI",
                             start="1944-01-07", end="2018-01-29",
                             interval="1d", group_by="ticker", 
                             auto_adjust=True, treads=True)

[*********************100%***********************]  4 of 4 completed


In [5]:
df_comp = raw_data.copy()

In [6]:
df_comp.head()

^N225                        ^GSPC                              \
            Open High Low Close Volume   Open   High    Low  Close Volume   
Date                                                                        
1944-01-07   NaN  NaN NaN   NaN    NaN  11.87  11.87  11.87  11.87    0.0   
1944-01-10   NaN  NaN NaN   NaN    NaN  11.85  11.85  11.85  11.85    0.0   
1944-01-11   NaN  NaN NaN   NaN    NaN  11.92  11.92  11.92  11.92    0.0   
1944-01-12   NaN  NaN NaN   NaN    NaN  11.84  11.84  11.84  11.84    0.0   
1944-01-13   NaN  NaN NaN   NaN    NaN  11.81  11.81  11.81  11.81    0.0   

           ^GDAXI                       ^FTSE                        
             Open High Low Close Volume  Open High Low Close Volume  
Date                                                                 
1944-01-07    NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  
1944-01-10    NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  
1944-01-11    NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  
1944-01-12    NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  
1944-01-13    NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN

In [7]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [9]:
df_comp = df_comp.iloc[1:]
df_comp.drop(['^GSPC', '^GDAXI', '^FTSE', '^N225'], axis=1, inplace=True)
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method='ffill')

# Creating Returns

In [10]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

# Splitting the Data

In [11]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size].copy(), df_comp.iloc[size:].copy()